In [ ]:
import json
from typing import List

import arxiv
from dotenv import load_dotenv


def search_papers(topic:str,max_results:int=5)->List[str]:
#     search papers and return papers content list
    return ["paper1", "paper2", "paper3"]

def extract_paper_info(paper_id:str)->str:
#     extract paper content by paper id
    return "This is the content of the paper with ID: " + paper_id


In [2]:
tools = [
    {
        "name":"search_papers",
        "description":"Search for papers on arXiv based on a topic and store their information.",
        "inout_schema":{
            "type":"object",
            "properties":{
                "topic":{
                    "type":"string",
                    "description":"the topic to search for"

                },
                "max_results":{
                    "type":"integer",
                    "description":"the maximum number of results to retrieve,default is 5.",
                    "default":5
                }
            }
        }
    },
    {
        "name":"extract_paper_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The paper ID to look for"
                }
            },
            "required": ["paper_id"]
        }
    }
]

In [ ]:
load_dotenv()
client = anthropic.Anthropic()





In [ ]:

tool_mapping_function = {
    "search_papers": search_papers,
    "extract_info": extract_paper_info
}

def execute_tool(tool_name:str,tool_args:dict)->str:
    tool_executed_result = tool_mapping_function[tool_name](**tool_args)
    result = ""
    if isinstance(tool_executed_result,list):
        result = tool_executed_result.join(", ")
    elif isinstance(tool_executed_result,dict):
        result = json.dumps(tool_executed_result)
    elif isinstance(tool_executed_result,str):
        result = tool_executed_result
    else:
        result = ""
    return result



In [ ]:
def process_query(query:str):
    messages = [
        {
            "role":"user",
            "content": query
        }
    ]
    response = client.messages.create(
        model="claude-2",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=1000,
        stop_sequences=["\n\n"]
    )

    _query = True

    while _query:
        assistant_content = []
        user_tools_content = []
        for content in response.content:
            if content.type == "text":
                print(content.text)
                assistant_content.append(content)
                if len(response.content) == 1:
                    _query = False
            elif content.type == "tool_use":
                assistant_content.append(content)

                too_name = content.name
                tool_id = content.id
                tool_args = content.input
                result = execute_tool(too_name,tool_args)
                user_tools_content.append({
                    "type":"tool_result",
                    "tool_use_id": tool_id,
                    "content": result
                })
        messages.append({
            "role":"assistant",
            "content": assistant_content
        })
        messages.append({
            "role":"user",
            "content": user_tools_content,
        })
        response = client.messages.create(model="claude-2",
                               tools=tools,
                               tool_choice="auto",
                               messages=messages)





In [ ]:
def chat_loop():
    while True:
        try:
            query = input("\nQuery:").strip()
            if query == "exit":
                break
            process_query(query)
        except Exception as e:
            print(e)
